# Wallet Feature Engineering

The purpose of this notebook is to...

# Read Data

Descitpion of the dataset ...

In [1]:
import os
import re
import datetime

data_dir = os.path.join(os.getcwd(), 'data')
cool_cats_nft_dir = os.path.join(data_dir, 'cool-cats-nft')

coolcats_wallets = [x for x in os.listdir(cool_cats_nft_dir) if re.search('^coolcatsnft_A\d\.xlsx', x)]
print(coolcats_wallets)

['coolcatsnft_A1.xlsx', 'coolcatsnft_A2.xlsx', 'coolcatsnft_A3.xlsx']


In [2]:
import pandas as pd


def read_combine(io):
    """
    Read a list of Excel files and combine them into a panadas DataFrame
    
    Args:
        io (list): list of fully qualitifed filenames
        
    Returns:
        DataFrame
    """
    
    list_df = (pd.read_excel(each) for each in io)
    return pd.concat(list_df)

In [44]:
import time

files = [os.path.join(cool_cats_nft_dir, x) for x in coolcats_wallets]
start_time = time.time()
wallets = read_combine(files)
total_time = time.time() - start_time
print("total minutes to load:", total_time / 60)

total minutes to load: 20.174154508113862


In [45]:
wallets = wallets[wallets.msg == "success"]

In [51]:
wallets.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2398450 entries, 0 to 399421
Data columns (total 32 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              2398450 non-null  int64  
 1   Unnamed: 1              2398450 non-null  int64  
 2   event_timestamp         2398450 non-null  object 
 3   event_type              2398450 non-null  object 
 4   token_id                2382796 non-null  object 
 5   num_sales               2382796 non-null  float64
 6   listing_time            2268796 non-null  object 
 7   token_owner_address     2382796 non-null  object 
 8   token_seller_address    2395806 non-null  object 
 9   deal_price              2398450 non-null  float64
 10  payment_token_symbol    2398411 non-null  object 
 11  payment_token_decimals  2398445 non-null  float64
 12  payment_token_usdprice  2397937 non-null  float64
 13  quantity                2394188 non-null  object 
 14  sta

In [52]:
wallets.drop(["Unnamed: 0", "Unnamed: 1", "starting_price", "ending_price",
              "approved_account", "bid_amount",
              "pages", "msg", "FILTER"], axis=1, inplace=True)

In [57]:
wallets.event_timestamp = pd.to_datetime(wallets.event_timestamp)
wallets.listing_time = pd.to_datetime(wallets.listing_time)

In [87]:
max(wallets.event_timestamp)

Timestamp('2022-05-10 23:35:55')

In [86]:
min(wallets.event_timestamp)

Timestamp('2017-07-04 04:33:49')

In [90]:
max(wallets.event_timestamp) - min(wallets.event_timestamp)

Timedelta('1771 days 19:02:06')

In [262]:
print("Total number of wallets:", wallets.wallet_address_input.nunique())

Total number of wallets: 9766


In [349]:
wallets.quantity.unique()

array(['1', '2', '4', '70', '3', '9', '10', '20', '5', '15', '6', '25',
       '50', '7', '8', '11', '12', '650', '14', '13', '19', '16', nan,
       '128', '100000000000', '18', '24', '22', '21', '17', '29', '35',
       '40', '28', '30', '100', '64', '32', '192', '420', '250', '51',
       '200', '69', '1000', '80', '172', '188', '55',
       '10000000000000000000000', '4000000000000000000', '27',
       '1000000000', '10000', '2500', '23', '49', '26', '247', '6000',
       '1000000000000000000', '12618', '8888', '87', '89', '10000000000',
       '10000000000000000000', '5000000000000000000', '31', '125',
       '43000000000', '61650000000', '25000000000',
       '612000000000000000000', '921', '300', '37', '34', '44', '33',
       '3579', '3000000000000000000', '60', '121', '390', '65', '400',
       '1000000000000000', '3000000000000000', '160000000',
       '100000000000000000', '100000000', '100000000000000000000',
       '500000000000000000000', '240', '255', 1.0, 5.0, 2.0, 7.0,

In [351]:
sum(wallets.quantity.isna())

4262

In [377]:
wallets.quantity=pd.to_numeric(wallets.quantity, errors='coerce')
sum(wallets.quantity.isna())

4262

In [382]:
wallets.quantile(q=[x / 1000 for x in range(0,1000, 10)])

,num_sales,deal_price,payment_token_decimals,payment_token_usdprice,quantity,block_number,is_private,deal_price_usd
0.00,0.0,0.000000e+00,0.0,5.011600e-11,1.0,3971224.00,0.0,0.000000
0.01,1.0,3.212983e+08,18.0,2.218410e+03,1.0,11817809.29,0.0,0.022499
0.02,1.0,5.000000e+15,18.0,2.249890e+03,1.0,12120872.98,0.0,12.005050
0.03,1.0,9.500000e+15,18.0,2.254960e+03,1.0,12434482.70,0.0,22.313492
0.04,1.0,1.000000e+16,18.0,2.254960e+03,1.0,12612300.28,0.0,23.825900
...,...,...,...,...,...,...,...,...
0.95,191.0,4.200000e+18,18.0,2.422550e+03,1.0,14597862.00,0.0,9659.160000
0.96,459.0,5.200000e+18,18.0,2.436340e+03,1.0,14621963.52,0.0,12005.050000
0.97,1124.0,6.890000e+18,18.0,2.436340e+03,1.0,14649170.13,0.0,15725.714117
0.98,2823.0,9.000000e+18,18.0,2.442540e+03,1.0,14681174.26,0.0,20799.630000


In [379]:
max(wallets.quantity)

1e+22

# Generate Features

## Impute data

_N.b._ The dataset is missing `winner_account_address` attribute to postively confirmed the buyer is
the same as the wallet owner, i.e. `wallet_address_input`. We therefore infer whether the wallet owner
is either the buyer or the seller as such:

In [393]:
import numpy as np

wallets.event_type = np.where(wallets.wallet_address_input == wallets.token_seller_address, "sell", "buy")

`duration` is the time between the token listed and the completion of the the sale.

In [389]:
wallets.rename(columns={"user_account_address": "token_seller_address"}, inplace=True)
wallets.duration = wallets.event_timestamp - wallets.listing_time
wallets.loc[:, ["token_seller_address", "event_type", "duration"]].head(10)

,token_seller_address,event_type,duration
0,0x82dc39052703cb51718b92fd62a6da6d1e749a0c,sell,0 days 09:45:26
1,0xd44a7b02e9692f491fb360d6a509e37c06bcd579,buy,0 days 00:56:41
2,0x56a7a519cb9d369334a24c98b44164d18a9b8385,buy,0 days 00:10:33
3,0x278d9db7032ffe25c5fcec6fb517f4e2041805d3,buy,0 days 10:08:26
4,0xef9fdc930d645299d01440d82b6c417cbd8f7162,buy,0 days 00:23:56
5,0xef9fdc930d645299d01440d82b6c417cbd8f7162,buy,0 days 00:07:53
6,0xd0d20158daa57b04c1094b7c0fa31efbdd675b52,buy,0 days 00:18:01
7,0x721b1b99af3ccbc2d42c1934f0aabc006ea36e31,buy,0 days 00:12:30
8,0x82dc39052703cb51718b92fd62a6da6d1e749a0c,sell,0 days 01:44:32
9,0x165dedca327cebac0a8222b71dc76a62b4727b83,buy,NaT


In [279]:
print(sum(wallets.payment_token_symbol.isna()), 
      sum(wallets.payment_token_decimals.isna()),
      sum(wallets.payment_token_usdprice.isna()))

39 5 513


In [282]:
wallets[wallets.payment_token_symbol.isna() |
        wallets.payment_token_decimals.isna() |
        wallets.payment_token_usdprice.isna()].shape[0]

513

is the total number of records missing either payment token symbol, token decimals, i.e. the deal price multiplication factor,
or the token to USD exchange rate. **We will ignore these record for now.**

In [402]:
wallets["deal_price_usd"] = wallets.deal_price / 10 ** wallets.payment_token_decimals * wallets.payment_token_usdprice
wallets.deal_price_usd.agg({max, np.mean, min})

max     6.925186e+06
min     0.000000e+00
mean    2.557588e+03
dtype: float64

In [401]:
wallets[wallets.quantity.isna()]["deal_price_usd"].quantile(q=[x / 10 for x in range(0, 10)])

0.0       0.000000
0.1      11.217255
0.2      19.093200
0.3      32.133298
0.4      47.733000
0.5      72.282328
0.6     115.448000
0.7     188.907200
0.8     377.833480
0.9    1158.180000
Name: deal_price_usd, dtype: float64

## WIP

將錢包地址分組

暫時給定任一個錢包地址來進行以下特徵計算 (最後再用迴圈串起來)

In [259]:
sectors = wallets.groupby("wallet_address_input")

df_temp3 = sectors.get_group("0x5338035c008ea8c4b850052bc8dad6a33dc2206c")
df_temp3 = df_temp3.reset_index(drop=True)  # Is this necessary?

In [260]:
df_temp3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5521 entries, 579808 to 585328
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   event_timestamp         5521 non-null   datetime64[ns] 
 1   event_type              5521 non-null   object         
 2   token_id                5503 non-null   object         
 3   num_sales               5503 non-null   float64        
 4   listing_time            5472 non-null   datetime64[ns] 
 5   token_owner_address     5503 non-null   object         
 6   token_seller_address    5521 non-null   object         
 7   deal_price              5521 non-null   float64        
 8   payment_token_symbol    5521 non-null   object         
 9   payment_token_decimals  5521 non-null   float64        
 10  payment_token_usdprice  5521 non-null   float64        
 11  quantity                5521 non-null   object         
 12  asset_bundle            18 

In [261]:
df_temp3.head()

,event_timestamp,event_type,token_id,num_sales,listing_time,token_owner_address,token_seller_address,deal_price,payment_token_symbol,payment_token_decimals,...,auction_type,transaction_hash,block_hash,block_number,is_private,duration,created_date,collection_slug,contract_address,wallet_address_input
579808,2022-05-04 03:29:18,successful,2977,1.0,2022-05-04 03:19:37,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0x4984fc170325e8fe57e9de1c2b74ce5eabb6f9da,8.450000e+17,ETH,18.0,...,NaN,0x5b7266357899fc13841a02456c8128b56e8852bc55ae...,0xe0e7e1a0a2516ec3ec8764e1f29773b9a77da8355ff3...,14708585.0,0.0,0 days 00:09:41,2022-05-04T03:29:50.900288,fragments-by-james-jean,0x7f268357a8c2552623316e2562d90e642bb538e5,0x5338035c008ea8c4b850052bc8dad6a33dc2206c
579809,2022-05-04 03:29:18,successful,3016,1.0,2022-05-04 00:14:49,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0xc293bc1602efeba837cb240c49476e1d3fe0fd98,8.450000e+17,ETH,18.0,...,NaN,0x5b7266357899fc13841a02456c8128b56e8852bc55ae...,0xe0e7e1a0a2516ec3ec8764e1f29773b9a77da8355ff3...,14708585.0,0.0,0 days 03:14:29,2022-05-04T03:29:50.461508,fragments-by-james-jean,0x7f268357a8c2552623316e2562d90e642bb538e5,0x5338035c008ea8c4b850052bc8dad6a33dc2206c
579810,2022-05-04 03:29:18,successful,4956,1.0,2022-05-03 19:39:05,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0xe2fb909159dea75b1520c382ca102989cdd1a276,8.400000e+17,ETH,18.0,...,NaN,0x5b7266357899fc13841a02456c8128b56e8852bc55ae...,0xe0e7e1a0a2516ec3ec8764e1f29773b9a77da8355ff3...,14708585.0,0.0,0 days 07:50:13,2022-05-04T03:29:50.009580,fragments-by-james-jean,0x7f268357a8c2552623316e2562d90e642bb538e5,0x5338035c008ea8c4b850052bc8dad6a33dc2206c
579811,2022-05-04 03:29:18,successful,5078,1.0,2022-05-04 01:40:56,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0x8a45d09b2dbbf1657fb8c14561b6525443631d22,8.500000e+17,ETH,18.0,...,NaN,0x5b7266357899fc13841a02456c8128b56e8852bc55ae...,0xe0e7e1a0a2516ec3ec8764e1f29773b9a77da8355ff3...,14708585.0,0.0,0 days 01:48:22,2022-05-04T03:29:49.627993,fragments-by-james-jean,0x7f268357a8c2552623316e2562d90e642bb538e5,0x5338035c008ea8c4b850052bc8dad6a33dc2206c
579812,2022-05-04 03:29:18,successful,5800,1.0,2022-05-04 02:52:43,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0xcc60f720388551bc9159cfed814a15de2f49d1e9,8.490000e+17,ETH,18.0,...,NaN,0x5b7266357899fc13841a02456c8128b56e8852bc55ae...,0xe0e7e1a0a2516ec3ec8764e1f29773b9a77da8355ff3...,14708585.0,0.0,0 days 00:36:35,2022-05-04T03:29:49.240455,fragments-by-james-jean,0x7f268357a8c2552623316e2562d90e642bb538e5,0x5338035c008ea8c4b850052bc8dad6a33dc2206c


In [9]:
import numpy as np

#買入價格，給定以太鏈。Eth = deal_price/(10**18)
df_temp3["cost"] = np.where(df_temp3["wallet_address_input"][0]==df_temp3["token_seller_address"], 0,df_temp3["deal_price"]/10**18)
#賣出價格
df_temp3["sellprice"] = np.where(df_temp3["wallet_address_input"][0]==df_temp3["token_seller_address"], df_temp3["deal_price"]/10**18, 0)

## cost and sellprice probably aren't necessary since there is the 'Buy_Sell' created below
## a deal_price_usd is recommended

#日期轉換
df_temp3["Datetime"] = pd.to_datetime(df_temp3["event_timestamp"]) # this can be store in event_timestamp instead of a new column
#買賣戳記
# Consider overwriting event_type column
df_temp3["Buy_Sell"] = np.where(df_temp3["wallet_address_input"][0]==df_temp3["token_seller_address"], "S", 'B')
#投資組合(庫存) << what do we plan to store here?
df_temp3["Profolio"] = np.NaN
#損益 << Profit? How do we plan to calcuate this for each row of event?
df_temp3["PL"] = 0
#token持有數量 << 
df_temp3["NFT_total_num"] = 0
#用collection_slug和tokenid組一個獨立欄位，用以紀錄錢包所持有的token
df_temp3["collection_slug_tokenid"] = df_temp3["collection_slug"] + df_temp3["token_id"]
#token從二級買進到賣出所持有的時間
df_temp3["HoldPeriod"] = np.NaN
df_temp3["Position"] = 0
df_temp3["Sell"] = 0

In [10]:
df_temp3.head()

,event_timestamp,event_type,token_id,num_sales,listing_time,token_owner_address,token_seller_address,deal_price,payment_token_symbol,payment_token_decimals,...,sellprice,Datetime,Buy_Sell,Profolio,PL,NFT_total_num,collection_slug_tokenid,HoldPeriod,Position,Sell
0,2022-05-04T03:29:18,successful,2977,1.0,2022-05-04T03:19:37,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0x4984fc170325e8fe57e9de1c2b74ce5eabb6f9da,8.450000e+17,ETH,18.0,...,0.0,2022-05-04 03:29:18,B,NaN,0,0,fragments-by-james-jean2977,NaN,0,0
1,2022-05-04T03:29:18,successful,3016,1.0,2022-05-04T00:14:49,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0xc293bc1602efeba837cb240c49476e1d3fe0fd98,8.450000e+17,ETH,18.0,...,0.0,2022-05-04 03:29:18,B,NaN,0,0,fragments-by-james-jean3016,NaN,0,0
2,2022-05-04T03:29:18,successful,4956,1.0,2022-05-03T19:39:05,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0xe2fb909159dea75b1520c382ca102989cdd1a276,8.400000e+17,ETH,18.0,...,0.0,2022-05-04 03:29:18,B,NaN,0,0,fragments-by-james-jean4956,NaN,0,0
3,2022-05-04T03:29:18,successful,5078,1.0,2022-05-04T01:40:56,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0x8a45d09b2dbbf1657fb8c14561b6525443631d22,8.500000e+17,ETH,18.0,...,0.0,2022-05-04 03:29:18,B,NaN,0,0,fragments-by-james-jean5078,NaN,0,0
4,2022-05-04T03:29:18,successful,5800,1.0,2022-05-04T02:52:43,0x5338035c008ea8c4b850052bc8dad6a33dc2206c,0xcc60f720388551bc9159cfed814a15de2f49d1e9,8.490000e+17,ETH,18.0,...,0.0,2022-05-04 03:29:18,B,NaN,0,0,fragments-by-james-jean5800,NaN,0,0


### Is the code block below attempting to calcuate the _current cumulative stat_ of each wallet?

In [11]:
porfolio_dict = {}#紀錄持有的NFT集合
porfolio_costdict = {}#紀錄買入成本
porfolio_datedict = {}#紀錄買入時間
count = 0
error = []
#資料時間是從新到舊，計算時要倒序，從舊到新去累計上來。
for i in range(len(df_temp3)-1,-1,-1):
    #初次買進NFT項目
    if df_temp3["collection_slug"][i] not in porfolio_dict.keys():
        if df_temp3["Buy_Sell"][i]=="B":
            #庫存加一
            count = count+1
            porfolio_dict[df_temp3["collection_slug"][i]] = [df_temp3["token_id"][i]]
            df_temp3.loc[i, "Profolio"] = [porfolio_dict]
            df_temp3.loc[i, "NFT_total_num"] = count
            #NFT成本
            porfolio_costdict[df_temp3["collection_slug_tokenid"][i]] = df_temp3["cost"][i]
            #NFT買進時間
            porfolio_datedict[df_temp3["collection_slug_tokenid"][i]] = df_temp3["Datetime"][i]
            #position
            df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())
            
        else:
            #賣出代表過去有可能發生來自於其他錢包轉移，但無法計算到先前持有的成本。
            df_temp3.loc[i, "NFT_total_num"] = count
            df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())
    else:
        #手上持有此項目的NFT
        if df_temp3["token_id"][i] not in porfolio_dict[df_temp3["collection_slug"][i]]:
            if df_temp3["Buy_Sell"][i]=="B":
                #買進加碼
                porfolio_dict[df_temp3["collection_slug"][i]].append(df_temp3["token_id"][i])
                df_temp3.loc[i, "Profolio"] = [porfolio_dict]
                #庫存加一
                count = count+1
                df_temp3.loc[i, "NFT_total_num"] = count
                #NFT成本
                porfolio_costdict[df_temp3["collection_slug_tokenid"][i]] = df_temp3["cost"][i]
                #NFT買進時間
                porfolio_datedict[df_temp3["collection_slug_tokenid"][i]] = df_temp3["Datetime"][i]
                #position
                df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())
                
            else:
                #賣出。有可能發生來自於其他錢包轉移，但無法計算到先前持有的成本。
                df_temp3.loc[i, "NFT_total_num"] = count
                df_temp3.loc[i, "Profolio"] = [porfolio_dict]
                df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())
        else:
            if df_temp3["Buy_Sell"][i]=="B":
                #不可能發生，因為tokenid是唯一的?
                df_temp3.loc[i, "NFT_total_num"] = count
                df_temp3.loc[i, "Profolio"] = [porfolio_dict]
                df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())
            else:
                #損益發生點，完成一次買入跟賣出
                #庫存減一
                count = count-1
                df_temp3.loc[i, "NFT_total_num"] = count
                #將token從porfolio移除
                porfolio_dict[df_temp3["collection_slug"][i]].remove(df_temp3["token_id"][i])
                df_temp3.loc[i, "Profolio"] = [porfolio_dict]
                if df_temp3["collection_slug_tokenid"][i] in porfolio_costdict.keys():
                    profit = df_temp3["sellprice"][i] - porfolio_costdict[df_temp3["collection_slug_tokenid"][i]]
                    df_temp3.loc[i, "PL"] =  profit        
                    #丟棄key and value因為賣出了
                    porfolio_costdict.pop(df_temp3["collection_slug_tokenid"][i])
                    df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())
                    #TOKEN從買入到賣出持有間隔時間
                    date_substrate = df_temp3["Datetime"][i] - porfolio_datedict[df_temp3["collection_slug_tokenid"][i]]
                    df_temp3.loc[i, "HoldPeriod"] =  date_substrate
                    #賣出戳記
                    df_temp3.loc[i, "Sell"] =  1
                    
                else:
                    #通常不會到這裡
                    error.append([df_temp3["wallet_address_input"][0],df_temp3["collection_slug_tokenid"][i]])
                    df_temp3.loc[i, "Profolio"] = [porfolio_dict]
                    df_temp3.loc[i, "Position"] = sum(porfolio_costdict.values())

In [12]:
#損益為正
def positive_SIGN(row):
    if row['PL_sign'] == 1:
        return 1
    return 0

#損益為負
def negative_SIGN(row):
    if row['PL_sign'] == -1 :
        return 1
    return 0

In [13]:
#累計損益是在一個錢包裡完成完整的買進賣出動作所累計的。
df_temp3['cum_PL'] = df_temp3.loc[::-1, 'PL'].cumsum()[::-1]
#總利潤
df_temp3['TotalRevenue'] = df_temp3['cum_PL'] - df_temp3["Position"]
#損益正負符號
df_temp3["PL_sign"] = np.sign(list(df_temp3["PL"].values))
#累計賣出數量
df_temp3["cum_Sell"] = df_temp3.loc[::-1, 'Sell'].cumsum()[::-1]
#損益為正做記號
df_temp3["positive_sign"] = df_temp3.apply(lambda row: positive_SIGN(row), axis=1)
#損益為負做記號
df_temp3["negative_sign"] = df_temp3.apply(lambda row: negative_SIGN(row), axis=1)
#累積正損益數
df_temp3["cum_positive_sign"] = df_temp3.loc[::-1, 'positive_sign'].cumsum()[::-1]
#累積負損益數
df_temp3["cum_negative_sign"] = df_temp3.loc[::-1, 'negative_sign'].cumsum()[::-1]
#勝率
df_temp3["winrate"] = df_temp3["cum_positive_sign"] / df_temp3['cum_Sell']
#輸錢率
df_temp3["lossrate"] = df_temp3["cum_negative_sign"] / df_temp3['cum_Sell']
#用0填補缺值
df_temp3["winrate"] = df_temp3["winrate"].fillna(0)
df_temp3["lossrate"] = df_temp3["lossrate"].fillna(0)
#接受問價而賣出做紀號
df_temp3["Bid_sell"] = np.where((df_temp3["payment_token_symbol"]=="WETH")&(df_temp3["Buy_Sell"]=="S"), 1,0)
#透過問價而買入做紀號
df_temp3["Bid_buy"] = np.where((df_temp3["payment_token_symbol"]=="WETH")&(df_temp3["Buy_Sell"]=="B"), 1,0)
#累計問價買入數
df_temp3["cum_Bid_buy"] = df_temp3.loc[::-1, 'Bid_buy'].cumsum()[::-1]
#累計接受問價賣出數
df_temp3["cum_Bid_sell"] = df_temp3.loc[::-1, 'Bid_sell'].cumsum()[::-1]

#勝率(透過問價而買入&接受問價而賣出)前者代表很會釣魚，後者代表失去信心或是無法抵抗高價誘惑
df_temp3["Bid_sell_rate"] = df_temp3["cum_Bid_sell"] / df_temp3["cum_Sell"]
df_temp3["Bid_sell_rate"] = df_temp3["Bid_sell_rate"].fillna(0)
df_temp3["Bid_buy_rate"] = df_temp3["cum_Bid_buy"] / df_temp3["NFT_total_num"]
#TOKEN賣出數/手上TOKEN持有數
df_temp3["sellposition_rate"] = df_temp3["cum_Sell"]/df_temp3["NFT_total_num"]

In [14]:
df_temp3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5521 entries, 0 to 5520
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   event_timestamp          5521 non-null   object        
 1   event_type               5521 non-null   object        
 2   token_id                 5503 non-null   object        
 3   num_sales                5503 non-null   float64       
 4   listing_time             5472 non-null   object        
 5   token_owner_address      5503 non-null   object        
 6   token_seller_address     5521 non-null   object        
 7   deal_price               5521 non-null   float64       
 8   payment_token_symbol     5521 non-null   object        
 9   payment_token_decimals   5521 non-null   float64       
 10  payment_token_usdprice   5521 non-null   float64       
 11  quantity                 5521 non-null   object        
 12  asset_bundle             18 non-nu

In [15]:
df_temp3.describe().loc["mean"]

num_sales                 1.017576e+02
deal_price                5.979868e+17
payment_token_decimals    1.800000e+01
payment_token_usdprice    2.390965e+03
block_number              1.335009e+07
is_private                1.279240e-03
cost                      1.693795e-01
sellprice                 4.286074e-01
PL                       -7.802300e-03
NFT_total_num             5.437930e+02
Position                  1.795545e+02
Sell                      9.871400e-02
cum_PL                    4.249531e+01
TotalRevenue             -1.370592e+02
PL_sign                   1.847491e-02
cum_Sell                  2.250926e+02
positive_sign             5.850389e-02
negative_sign             4.002898e-02
cum_positive_sign         1.614360e+02
cum_negative_sign         6.358920e+01
winrate                   7.560965e-01
lossrate                  2.435963e-01
Bid_sell                  8.694077e-03
Bid_buy                   1.811266e-04
cum_Bid_buy               2.807462e-01
cum_Bid_sell             

## WIP

In [16]:
wallets.token_seller_address.nunique()

215099

In [37]:
sum(wallets.payment_token_symbol.isna())

39

In [391]:
wallets.quantity.unique()

array([1.00000000e+00, 2.00000000e+00, 4.00000000e+00, 7.00000000e+01,
       3.00000000e+00, 9.00000000e+00, 1.00000000e+01, 2.00000000e+01,
       5.00000000e+00, 1.50000000e+01, 6.00000000e+00, 2.50000000e+01,
       5.00000000e+01, 7.00000000e+00, 8.00000000e+00, 1.10000000e+01,
       1.20000000e+01, 6.50000000e+02, 1.40000000e+01, 1.30000000e+01,
       1.90000000e+01, 1.60000000e+01,            nan, 1.28000000e+02,
       1.00000000e+11, 1.80000000e+01, 2.40000000e+01, 2.20000000e+01,
       2.10000000e+01, 1.70000000e+01, 2.90000000e+01, 3.50000000e+01,
       4.00000000e+01, 2.80000000e+01, 3.00000000e+01, 1.00000000e+02,
       6.40000000e+01, 3.20000000e+01, 1.92000000e+02, 4.20000000e+02,
       2.50000000e+02, 5.10000000e+01, 2.00000000e+02, 6.90000000e+01,
       1.00000000e+03, 8.00000000e+01, 1.72000000e+02, 1.88000000e+02,
       5.50000000e+01, 1.00000000e+22, 4.00000000e+18, 2.70000000e+01,
       1.00000000e+09, 1.00000000e+04, 2.50000000e+03, 2.30000000e+01,
      

In [390]:
wallets[~wallets.payment_token_symbol.isna()].quantity.unique()

array([1.0000e+00, 2.0000e+00, 4.0000e+00, 7.0000e+01, 3.0000e+00,
       9.0000e+00, 1.0000e+01, 2.0000e+01, 5.0000e+00, 1.5000e+01,
       6.0000e+00, 2.5000e+01, 5.0000e+01, 7.0000e+00, 8.0000e+00,
       1.1000e+01, 1.2000e+01, 6.5000e+02, 1.4000e+01, 1.3000e+01,
       1.9000e+01, 1.6000e+01,        nan, 1.2800e+02, 1.0000e+11,
       1.8000e+01, 2.4000e+01, 2.2000e+01, 2.1000e+01, 1.7000e+01,
       2.9000e+01, 3.5000e+01, 4.0000e+01, 2.8000e+01, 3.0000e+01,
       1.0000e+02, 6.4000e+01, 3.2000e+01, 1.9200e+02, 4.2000e+02,
       2.5000e+02, 5.1000e+01, 2.0000e+02, 6.9000e+01, 1.0000e+03,
       8.0000e+01, 1.7200e+02, 1.8800e+02, 5.5000e+01, 1.0000e+22,
       4.0000e+18, 2.7000e+01, 1.0000e+09, 1.0000e+04, 2.5000e+03,
       2.3000e+01, 4.9000e+01, 2.6000e+01, 2.4700e+02, 6.0000e+03,
       1.0000e+18, 1.2618e+04, 8.8880e+03, 8.7000e+01, 8.9000e+01,
       1.0000e+10, 1.0000e+19, 5.0000e+18, 3.1000e+01, 1.2500e+02,
       4.3000e+10, 6.1650e+10, 2.5000e+10, 6.1200e+20, 9.2100e

In [392]:
wallets[wallets.quantity.isna()]

,event_timestamp,event_type,token_id,num_sales,listing_time,token_owner_address,token_seller_address,deal_price,payment_token_symbol,payment_token_decimals,...,transaction_hash,block_hash,block_number,is_private,duration,created_date,collection_slug,contract_address,wallet_address_input,deal_price_usd
97801,2020-06-27 19:12:07,buy,147226,1.0,NaT,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,0x42a60d2f2ffa2150c568010a8d425f0aad284fd2,3.249210e+16,ETH,18.0,...,0x2ff432a6205be89a625a7016815b5a81991541d94173...,0x0c2cd5d23b6676863b2278e225ed650f4c0884860c4b...,10349619.0,NaN,NaT,2020-06-27T19:12:48.763876,axie,0xf4985070ce32b6b1994329df787d1acc9a2dd9e2,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,75.729669
97802,2020-06-20 16:16:52,buy,44584,2.0,NaT,0x6d1f5fac38edca69b9d02637a173c1e62d331896,0x8b3ad493c077e894a034db7eb53e8285560298fd,2.450000e+16,ETH,18.0,...,0x21746dd417132bb6e3b6ca77c809ba62322302ff16ea...,0x7124a292f83004a5f888acd3d808a4e6544c94568534...,10303653.0,NaN,NaT,2020-06-20T16:17:15.773754,axie,0xf4985070ce32b6b1994329df787d1acc9a2dd9e2,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,57.102395
97803,2020-06-19 17:01:06,buy,140164,4.0,NaT,0xc301878610b1952e94a57d9958fba4ec043537e4,0x9b7061023cd42263448d48c48572507f19f39b78,7.500000e+16,ETH,18.0,...,0xb0993b84033070e3b6e11c10e55ffcc78386f287188c...,0xc08102bade64f5976a023213267cc09b06dc714a5df4...,10297405.0,NaN,NaT,2020-06-19T17:01:53.569908,axie,0xf4985070ce32b6b1994329df787d1acc9a2dd9e2,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,174.803250
97804,2020-06-19 16:56:36,buy,71500,3.0,NaT,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,0x937b9093cd9d8798930e394d188f9b5596d49f54,5.000000e+16,ETH,18.0,...,0x97ce92af142c265957c66c5db4909a66d282b39e2df6...,0xe42efeaef744ded41ffd2d35a40eccef40a87c89a98b...,10297384.0,NaN,NaT,2020-06-19T16:57:06.608060,axie,0xf4985070ce32b6b1994329df787d1acc9a2dd9e2,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,116.535500
97805,2020-06-19 16:52:38,buy,121128,2.0,NaT,0x7a9112792211461205db4191381866b0508fa4a8,0xb4d055d63cc6a7bfb51a588ddaeb245ce5e3fc48,1.890000e+16,ETH,18.0,...,0xeb9f95b93ee31757b257ba06a30d35a56bb0de35a511...,0xf0237c4fada0f5fb6fdde8c1ba8702bac7cf1598ea44...,10297361.0,NaN,NaT,2020-06-19T16:52:55.760877,axie,0xf4985070ce32b6b1994329df787d1acc9a2dd9e2,0x3bd77b00f02c8bcff586c565e2c5e6b6c5878ec3,44.050419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169103,2019-05-20 01:12:22,buy,23300,1.0,2019-05-20 00:09:28,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,0x05f9bbbd6af1699cb3ca8c14bd38b2b47bd6b2ec,1.000000e+15,ETH,18.0,...,0x37e0c7d0b3b606c2dd9ec83fdf3c1c6aff70e02b60d8...,"b""t`\x91E\x9d\x96;\x9e\xe3\xd6y'\xfaO\xca\xbe\...",7793907.0,0.0,0 days 01:02:54,2019-05-20T01:12:56.975962,neon-district,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,2.382100
169104,2019-05-20 01:12:07,buy,32194,1.0,2019-05-20 00:11:56,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,0x05f9bbbd6af1699cb3ca8c14bd38b2b47bd6b2ec,1.000000e+15,ETH,18.0,...,0x0362ec6a23c7cc9aff6ca78587ffaa09bf8708ad51cd...,"b""w\xb2\xa2\xecN\xe2\xf4\x10N\xd8\xfc\xa5\x8d'...",7793905.0,0.0,0 days 01:00:11,2019-05-20T01:12:31.755591,neon-district,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,2.382100
169105,2019-05-20 01:10:19,buy,23301,1.0,2019-05-20 00:09:59,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,0x05f9bbbd6af1699cb3ca8c14bd38b2b47bd6b2ec,1.000000e+15,ETH,18.0,...,0x55145bccaeed167e32804c9b2af89a859993ddba6598...,b'\x06B\x1c7D\xcd\x89\xd1\xbe\xcb\xf1\xd5~?v\x...,7793900.0,0.0,0 days 01:00:20,2019-05-20T01:10:57.798340,neon-district,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,2.382100
169106,2019-05-20 01:09:42,buy,33912,1.0,2019-05-20 00:16:54,0xa5a0b7c3dd5dddbfbd51e56b9170bb6d1253788b,0x05f9bbbd6af1699cb3ca8c14bd38b2b47bd6b2ec,1.000000e+15,ETH,18.0,...,0x613a7d28e158d5db005e808dbd4d152cbf0ed8968b68...,b'4>S\xda\xff\x15\x02\xe5\x1dP=\xaa\xfb{]\x

## Create features

In [263]:
wallets

,event_timestamp,event_type,token_id,num_sales,listing_time,token_owner_address,token_seller_address,deal_price,payment_token_symbol,payment_token_decimals,...,auction_type,transaction_hash,block_hash,block_number,is_private,duration,created_date,collection_slug,contract_address,wallet_address_input
0,2022-05-07 13:20:01,successful,13921,1.0,2022-05-07 03:34:35,0x82a4d882e5a62926c2a330b17dbb972d34a7d730,0x82dc39052703cb51718b92fd62a6da6d1e749a0c,4.500000e+18,ETH,18.0,...,NaN,0x0c0161eb69be05977e4d333bb7bc096296a3988eaa85...,0xb0d9324f1bbc40d7d80b2c50be240c7057a056e3b4b6...,14730037.0,0.0,0 days 09:45:26,2022-05-07T13:20:33.224540,otherdeed,0x7f268357a8c2552623316e2562d90e642bb538e5,0x82dc39052703cb51718b92fd62a6da6d1e749a0c
1,2022-05-07 09:03:02,successful,562954248415769,4.0,2022-05-07 08:06:21,0x82dc39052703cb51718b92fd62a6da6d1e749a0c,0xd44a7b02e9692f491fb360d6a509e37c06bcd579,1.100000e+18,ETH,18.0,...,NaN,0x42aa8b92b0b75ef305bb54dc001a7794952ad16b0ef0...,0x6de5b2cde824403911e03d08e2d0055250bf0d182c2f...,14728870.0,0.0,0 days 00:56:41,2022-05-07T09:03:17.428510,10ktf,0x7f268357a8c2552623316e2562d90e642bb538e5,0x82dc39052703cb51718b92fd62a6da6d1e749a0c
2,2022-05-07 03:28:14,successful,281479271685666,2.0,2022-05-07 03:17:41,0xc876b346d50c4199458ee0e3754883f62fe3a5b0,0x56a7a519cb9d369334a24c98b44164d18a9b8385,1.000000e+18,ETH,18.0,...,NaN,0xa5ca52af5403d75a89fb6f4e99c102978b3426135786...,0x77ec903db1a952735bf4279e37122b7a5cf1773801a8...,14727463.0,0.0,0 days 00:10:33,2022-05-07T03:28:49.898598,10ktf,0x7f268357a8c2552623316e2562d90e642bb538e5,0x82dc39052703cb51718b92fd62a6da6d1e749a0c
3,2022-05-06 11:45:23,successful,1022766819668093232954669218231971723193525699...,1.0,2022-05-06 01:36:57,0x82dc39052703cb51718b92fd62a6da6d1e749a0c,0x278d9db7032ffe25c5fcec6fb517f4e2041805d3,9.800000e+16,ETH,18.0,...,NaN,0xee59e916d36f28bb32046e6f0793a00ba8b05431511a...,0x25bfefeb0e407cd689a1eeb610322ec39d5ec45a4a21...,14723351.0,0.0,0 days 10:08:26,2022-05-06T11:45:37.683956,ens,0x7f268357a8c2552623316e2562d90e642bb538e5,0x82dc39052703cb51718b92fd62a6da6d1e749a0c
4,2022-05-05 03:36:13,successful,7588,1.0,2022-05-05 03:12:17,0x82dc39052703cb51718b92fd62a6da6d1e749a0c,0xef9fdc930d645299d01440d82b6c417cbd8f7162,2.400000e+17,ETH,18.0,...,NaN,0x2a36510d98f0012e4a20065443c21a8f5d45bb2cc26a...,0x4f98adee98aeaa2e454e7bf4e2afe3a59ff2a1851afd...,14714924.0,0.0,0 days 00:23:56,2022-05-05T03:36:43.828893,somethingtoken,0x7f268357a8c2552623316e2562d90e642bb538e5,0x82dc39052703cb51718b92fd62a6da6d1e749a0c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399417,2021-10-11 22:22:25,successful,5008,1.0,2021-10-11 17:39:07,0x899f7e7bba83a4462144e576f8f14f018bb30d2a,0x4de910a6ca7cec4fe0db9edc24c3a66d6558ea3f,2.490000e+17,ETH,18.0,...,NaN,0x81a0fce503c7b5f7118023b0b9f0beacfe2cb31a56f8...,0xe9d9ff912e05d98f4d03f7127591b4acbd6a77ce36b1...,13399941.0,0.0,0 days 04:43:18,2021-10-11T22:22:56.482359,eponym,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0x899f7e7bba83a4462144e576f8f14f018bb30d2a
399418,2021-10-11 20:04:51,successful,6886,1.0,2021-10-11 17:51:29,0x899f7e7bba83a4462144e576f8f14f018bb30d2a,0xf9d681c3b81aa1d0ecb3fdb4c69ca57714eb63f4,1.500000e+17,ETH,18.0,...,NaN,0xb3ed887507415a331fb98e52d6a602f4ae03c199011c...,0xfe4f306ddd30d4a243874161cb98af2b47063ef032fe...,13399312.0,0.0,0 days 02:13:22,2021-10-11T20:05:14.211280,eponym,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0x899f7e7bba83a4462144e576f8f14f018bb30d2a
399419,2021-10-11 17:45:54,successful,8855789430591774980265252005795383506440366856...,1.0,2021-10-08 14:18:44,0x0000000000000000000000000000000000000000,0xc3c9fdee83ad8c7b29b5ce2c6b8d19fa116c0e74,1.000000e+17,ETH,18.0,...,NaN,0x02ea671495300f644c162a028eb0b94ed70a5d3f5e55...,0x08c62166483e3189e7faebd4fc647a6cd1796fe99425...,13398700.0,0.0,3 days 03:27:10,2021-10-11T17:46:35.187753,legendz,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0x899f7e7bba83a4462144e576f8f14f018bb30d2a
399420,2021-10-10 00:51:27,successful,8953,3

### Wallet age (WIP)

In [386]:
df=wallets.groupby("wallet_address_input") \
    .agg({"event_timestamp": [max, min]}) \
    .assign(wallet_age=lambda x : x.loc[:, ("event_timestamp", "max")] - x.loc[:, ("event_timestamp", "min")])

In [222]:
df.columns.set_levels([['event_timestamp', ""], ['max', 'min', 'wallet_age']], level=[0,1])

MultiIndex([('event_timestamp',        'max'),
            ('event_timestamp',        'min'),
            (               '', 'wallet_age')],
           )

In [249]:
df.droplevel(0, axis=1)

,max,min,
wallet_address_input,,,
0x000000000ad266ec3db44bbe580e87f9baa358e6,2022-05-02 00:53:24,2022-01-19 23:25:39,102 days 01:27:45
0x000000070f91b6c56fa08d4f3a26c7fc992b38f4,2022-05-02 15:54:00,2021-10-14 23:22:26,199 days 16:31:34
0x0004ff7e7217dc672874fece2c7588581e97b1a7,2022-02-12 11:32:12,2021-08-23 11:14:45,173 days 00:17:27
0x000cd27f10dffac73201258eaa3925c0452051a0,2022-05-05 20:17:48,2021-05-04 14:29:13,366 days 05:48:35
0x000f9aa9783be4d2955faca0f9a4d3c676fc9e0b,2022-04-19 13:33:12,2021-08-01 05:59:49,261 days 07:33:23
...,...,...,...
0xffea8516005b365fe9cb43393c74c3965c98c164,2022-03-11 14:28:18,2021-02-13 20:17:49,390 days 18:10:29
0xffeafbc24c7f8736d682f49d1bf3b023a39aa737,2022-05-04 22:39:24,2021-07-29 16:26:06,279 days 06:13:18
0xfffa6fc6acc3dbe04b175862376f1c5ff88cf9c1,2022-05-10 00:11:28,2022-01-26 12:46:15,103 days 11:25:13


In [383]:
df.reset_index(col_level=1).droplevel(0, axis=1).assign(wallet_age=lambda x: x["max"]-x["min"])

,wallet_account_address,max,min,,wallet_age
0,0x000000000000123ca35c69ba3f852a46b2a27c94,2022-02-17 09:37:34,2022-02-17 09:08:33,0 days 00:29:01,0 days 00:29:01
1,0x0000000000015b23c7e20b0ea5ebd84c39dcbe60,2021-07-26 15:47:53,2021-06-07 23:09:17,48 days 16:38:36,48 days 16:38:36
2,0x00000000000360176d958e11c140308cd0863679,2021-09-09 19:10:47,2021-08-06 06:43:27,34 days 12:27:20,34 days 12:27:20
3,0x000000000004d7463d0f9c77383600bc82d612f5,2021-11-14 09:27:10,2021-11-14 09:27:10,0 days 00:00:00,0 days 00:00:00
4,0x00000000000a486c964069bb7390ae37010a04ca,2022-05-03 01:38:31,2022-04-18 23:53:59,14 days 01:44:32,14 days 01:44:32
...,...,...,...,...,...
215094,0xffffe59e4ebefce216470864fd92407023288cb4,2022-01-30 09:43:45,2022-01-30 09:43:45,0 days 00:00:00,0 days 00:00:00
215095,0xffffe96d5df4b535022bcf9a901716ba3ebd8a82,2022-02-06 21:14:21,2022-01-06 15:05:31,31 days 06:08:50,31 days 06:08:50
215096,0xfffff6e70842330948ca47254f2be673b1cb0db7,2021-06-30 17:05:50,2021-06-29 22:57:27,0 days 18:08:23,0 days 18:08:23
215097,0xffffff5800b709071d4adc74759ae4b89bef2a9d,2022-05-04 17:03:44,2022-04-19 04:24:04,15 days 12:39:40,15 days 12:39:40


## Transform dataset

In [313]:
wallets.wallet_address_input.nunique()

9766

In [366]:
wallets.columns

Index(['event_timestamp', 'event_type', 'token_id', 'num_sales',
       'listing_time', 'token_owner_address', 'user_account_address',
       'deal_price', 'payment_token_symbol', 'payment_token_decimals',
       'payment_token_usdprice', 'quantity', 'asset_bundle', 'auction_type',
       'transaction_hash', 'block_hash', 'block_number', 'is_private',
       'duration', 'created_date', 'collection_slug', 'contract_address',
       'wallet_address_input', 'deal_price_usd'],
      dtype='object')

In [365]:
wallets.rename(columns={"token_seller_address": "user_account_address"}, inplace=True)
wallets.user_account_address.nunique()

215099

In [367]:
wallets.groupby(["user_account_address", "event_type"]).sum()

,,num_sales,deal_price,payment_token_decimals,payment_token_usdprice,block_number,is_private,deal_price_usd
user_account_address,event_type,,,,,,,
0x000000000000123ca35c69ba3f852a46b2a27c94,buy,3.0,1.479000e+18,36.0,4773.25,28445621.0,0.0,3536.472260
0x0000000000015b23c7e20b0ea5ebd84c39dcbe60,buy,10.0,3.730000e+18,36.0,4737.92,25493004.0,0.0,8785.110700
0x00000000000360176d958e11c140308cd0863679,buy,16799.0,4.304900e+18,216.0,28500.43,156312940.0,0.0,10278.447125
0x000000000004d7463d0f9c77383600bc82d612f5,buy,2.0,2.500000e+18,18.0,2249.89,13613175.0,0.0,5624.725000
0x00000000000a486c964069bb7390ae37010a04ca,buy,3.0,5.440000e+16,36.0,4731.51,29313998.0,0.0,128.946970
...,...,...,...,...,...,...,...,...
0xffffe59e4ebefce216470864fd92407023288cb4,buy,3.0,4.790000e+17,18.0,2392.29,14106408.0,0.0,1145.906910
0xffffe96d5df4b535022bcf9a901716ba3ebd8a82,buy,4.0,2.330000e+18,36.0,4713.89,28107432.0,0.0,5460.431400
0xfffff6e70842330948ca47254f2be673b1cb0db7,buy,6.0,9.195000e+18,54.0,7077.25,38203705.0,0.0,22076.943750


In [359]:
wallets[wallets.user_account_address == "0x000000000000123ca35c69ba3f852a46b2a27c94"]

,event_timestamp,event_type,token_id,num_sales,listing_time,token_owner_address,user_account_address,deal_price,payment_token_symbol,payment_token_decimals,...,transaction_hash,block_hash,block_number,is_private,duration,created_date,collection_slug,contract_address,wallet_address_input,deal_price_usd
86945,2022-02-17 09:37:34,buy,7527,1.0,2022-02-15 20:00:50,0xd311bdacb151b72bddfee9cbdc414af22a5e38dc,0x000000000000123ca35c69ba3f852a46b2a27c94,6.800000e+17,ETH,18.0,...,0xabd7bba058aceebb37d724fb0875579ef7e447884289...,0xb726498fab138863e12d88abf217b70b1facdc69c2b4...,14222873.0,0.0,1 days 13:36:44,2022-02-17T09:37:47.808970,raidpartyfighters,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0x4bdc1cad2d045ec17955688d0fefeb99a385c30f,1584.88280
278745,2022-02-17 09:08:33,buy,354,2.0,2022-02-15 20:01:20,0xd311bdacb151b72bddfee9cbdc414af22a5e38dc,0x000000000000123ca35c69ba3f852a46b2a27c94,7.990000e+17,ETH,18.0,...,0x26549fd9bf01563258e633a6c6d0239718cff37f35a8...,0xfce46d606649b8608f9e1b31e3ee6de18d157fbd8d05...,14222748.0,0.0,1 days 13:07:13,2022-02-17T09:08:52.893135,raidpartyfighters,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0x150fa2afc4db393b4d231cbace82ecfc7d3b4be9,1951.58946


In [360]:
df=wallets.groupby("user_account_address") \
    .agg({"event_timestamp": [max, min], "quantity": [sum]}) \
    .assign(wallet_age=df.loc[:, ("event_timestamp", "max")] - df.loc[:, ("event_timestamp", "min")])
df.sort_values(["wallet_age"], ascending=False)

TypeError: can only concatenate str (not "float") to str

In [331]:
x.loc[:, ("event_timestamp", "max")] - x.loc[:, ("event_timestamp", "min")]

wallet_account_address
0x000000000000123ca35c69ba3f852a46b2a27c94    0 days 00:29:01
0x0000000000015b23c7e20b0ea5ebd84c39dcbe60   48 days 16:38:36
0x00000000000360176d958e11c140308cd0863679   34 days 12:27:20
0x000000000004d7463d0f9c77383600bc82d612f5    0 days 00:00:00
0x00000000000a486c964069bb7390ae37010a04ca   14 days 01:44:32
                                                   ...       
0xffffe59e4ebefce216470864fd92407023288cb4    0 days 00:00:00
0xffffe96d5df4b535022bcf9a901716ba3ebd8a82   31 days 06:08:50
0xfffff6e70842330948ca47254f2be673b1cb0db7    0 days 18:08:23
0xffffff5800b709071d4adc74759ae4b89bef2a9d   15 days 12:39:40
0xffffffbf3bf3b043a7457e78b4a5313a58dc5d14    0 days 00:00:00
Length: 215099, dtype: timedelta64[ns]

# Explore Data

In [297]:
wallets.groupby(["wallet_address_input", "event_type"]).sum()

num_sales  \
wallet_address_input                       event_type              
0x000000000ad266ec3db44bbe580e87f9baa358e6 sell             17.0   
0x000000070f91b6c56fa08d4f3a26c7fc992b38f4 buy              63.0   
                                           sell             17.0   
0x0004ff7e7217dc672874fece2c7588581e97b1a7 buy              56.0   
                                           sell             36.0   
...                                                          ...   
0xfffa6fc6acc3dbe04b175862376f1c5ff88cf9c1 sell          63515.0   
0xfffcde6975788ff3c5ad6bc5d00c3fbc9d42483c buy            2010.0   
                                           sell             10.0   
0xffffc32855b2620c86f413065af8c58ec68d474d buy            1246.0   
                                           sell            757.0   

                                                         deal_price  \
wallet_address_input                       event_type                 
0x000000000ad266ec3db44bbe580e87f9baa358e6 sell        1.695600e+19   
0x000000070f91b6c56fa08d4f3a26c7fc992b38f4 buy         2.879600e+19   
                                           sell        3.778000e+19   
0x0004ff7e7217dc672874fece2c7588581e97b1a7 buy         3.482230e+19   
                                           sell        3.590000e+19   
...                                                             ...   
0xfffa6fc6acc3dbe04b175862376f1c5ff88cf9c1 sell        4.801470e+20   
0xfffcde6975788ff3c5ad6bc5d00c3fbc9d42483c buy         9.914000e+18   
                                           sell        9.099000e+18   
0xffffc32855b2620c86f413065af8c58ec68d474d buy         2.850000e+17   
                                           sell        3.721950e+20   

                                                       payment_token_decimals  \
wallet_address_input                       event_type                           
0x000000000ad266ec3db44bbe580e87f9baa358e6 sell                         108.0   
0x000000070f91b6c56fa08d4f3a26c7fc992b38f4 buy                          612.0   
                                           sell                          90.0   
0x0004ff7e7217dc672874fece2c7588581e97b1a7 buy                          378.0   
                                           sell                         216.0   
...                                                                       ...   
0xfffa6fc6acc3dbe04b175862376f1c5ff88cf9c1 sell                         756.0   
0xfffcde6975788ff3c5ad6bc5d00c3fbc9d42483c buy                           90.0   
                                           sell                          90.0   
0xffffc32855b2620c86f413065af8c58ec68d474d buy                           36.0   
                                           sell                         720.0   

                                                       payment_token_usdprice  \
wallet_address_input                       event_type                           
0x000000000ad266ec3db44bbe580e87f9baa358e6 sell                      14496.10   
0x000000070f91b6c56fa08d4f3a26c7fc992b38f4 buy                       76496.26   
                                           sell                      11249.45   
0x0004ff7e7217dc672874fece2c7588581e97b1a7 buy                       47471.93   
                                           sell                      27142.56   
...                                                                       ...   
0xfffa6fc6acc3dbe04b175862376f1c5ff88cf9c1 sell                     100048.20   
0xfffcde6975788ff3c5ad6bc5d00c3fbc9d42483c buy                       11806.70   
                                           sell                      11806.70   
0xffffc32855b2620c86f413065af8c58ec68d474d buy                        4802.02   
                                           sell                      96040.64   

                                                       block_number  \
wallet_address_input                       event_type              

# Note
Feature engineering in ML
1. Feature Creation
1. Transformations
1. Feature Extraction
1. and Feature Selection.